In [1]:
import pandas as pd
from openpyxl import load_workbook
import openpyxl
import numpy as np
from datetime import date, timedelta

In [2]:
def cell_to_num(char):
    char = char.upper()
    if char.isalpha():
        num = ord(char) - 64
    else:
        num = 0
    return num
print(cell_to_num('a'))

1


In [3]:
"""
# Windows file system
first_file = 'C:/Users/panka/Downloads/Nozzle Sales Report.xlsx'
second_file = 'C:/Users/panka/Downloads/Nozzle Sales Report (1).xlsx'
"""
# linux file system
first_file = '/mnt/c/Users/panka/Downloads/Nozzle Sales Report.xlsx'
second_file = '/mnt/c/Users/panka/Downloads/Nozzle Sales Report (1).xlsx'

wb_check = load_workbook(first_file)

/home/pankaj/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:214: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
def data_cleaning(path):
    wb = load_workbook(path)
    ws = wb.active
    
    #Unmerge cell
    ws.unmerge_cells('O1:P1')
    ws.unmerge_cells('Q1:R1')
    ws.unmerge_cells('S1:T1')
    ws.unmerge_cells('U1:V1')
    if path[-7] == '1':
        #renaming issue_sale value with dates
        ws['S1'] = str(date.today() - timedelta(days=4))
        ws['T1'] = str(date.today() - timedelta(days=3))
    else:
        ws['S1'] = str(date.today() - timedelta(days=2))    
        ws['T1'] = str(date.today() - timedelta(days=1))
    
    print(ws['S1'].value)
    print(ws['T1'].value)    
    
    #Delete column U:V
    ws.delete_cols(cell_to_num('U'), cell_to_num('V')-cell_to_num('U')+1)
    #Delete column O:R
    ws.delete_cols(cell_to_num('O'), cell_to_num('R')-cell_to_num('O')+1)
    #Delete column K:M
    ws.delete_cols(cell_to_num('K'), cell_to_num('M')-cell_to_num('K')+1)
    #Delete column B:F
    ws.delete_cols(cell_to_num('B'), cell_to_num('F')-cell_to_num('B')+1)
    #Delete second row
    ws.delete_rows(2)
    wb.save(path)

In [5]:
def create_pivot(path):
    #reading excel file using pandas
    old_df = pd.read_excel(path)
    current_wb = load_workbook(path)
    ws = current_wb.active
    
    #filter out the 'NA' value in the product column
    df = old_df[
        (old_df['Product'] == 'XP') |
        (old_df['Product'] == 'HS') |
        (old_df['Product'] == 'MS')
    ]
    
    df.loc[df['Product'] == 'XP', 'Product'] = 'MS'
    
    pvt_all = pd.pivot_table(df,
                            index=["SA"],
                            columns=['Product'],
                            values=[ws['H1'].value, ws['G1'].value],
                            aggfunc=np.sum)
    return pvt_all

In [6]:
data_cleaning(first_file)
data_cleaning(second_file)

2020-07-26
2020-07-27
2020-07-24
2020-07-25


In [7]:
pvt = pd.concat([create_pivot(first_file), create_pivot(second_file)], axis=1)
print(pvt)

/home/pankaj/.local/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


                      2020-07-26            2020-07-27            2020-07-24  \
Product                       HS        MS          HS        MS          HS   
SA                                                                             
Fuels- Bargarh(R)      77143.780  26767.45   75268.595  46382.57   79148.060   
Fuels-Bhawanipatna(R   85663.850  34282.90   98617.470  48384.95  100737.950   
Fuels-JEYPORE(R)       88960.590  37026.94  116454.360  77550.07  143278.100   
Fuels-Jharsuguda(R)    89383.740  23636.54  123893.000  35085.53  105291.880   
Fuels-ROURKELA(R)     264787.380  33459.79  297054.340  38585.19  280507.700   
Fuels-SAMBALPUR (R)   108428.850  22160.51  113849.340  37967.95  114771.750   
Keonjhar I RSA        101078.864  11532.63  105566.910  20263.91  115044.714   
Keonjhar II RSA        11945.410   4756.00   21604.370   7816.54   13784.780   

                                2020-07-25            
Product                     MS          HS        MS  
SA       

In [8]:
with pd.ExcelWriter(first_file, engine='openpyxl') as writer:
    writer.book = load_workbook(first_file)
    pvt.to_excel(writer, 'pivot sheet', index=True)

In [9]:
df = pd.read_excel(first_file, sheet_name='pivot sheet')
print(df.columns.values)

['Unnamed: 0' '2020-07-26' 'Unnamed: 2' '2020-07-27' 'Unnamed: 4'
 '2020-07-24' 'Unnamed: 6' '2020-07-25' 'Unnamed: 8']


In [10]:
cols = df.columns.values
order = [0, 3, 4, 1, 2, 7, 8, 5, 6]
cols = [cols[i] for i in order]
print(cols)

['Unnamed: 0', '2020-07-27', 'Unnamed: 4', '2020-07-26', 'Unnamed: 2', '2020-07-25', 'Unnamed: 8', '2020-07-24', 'Unnamed: 6']


In [11]:
df = df[cols]
print(df)

             Unnamed: 0 2020-07-27 Unnamed: 4 2020-07-26 Unnamed: 2  \
0               Product         HS         MS         HS         MS   
1                    SA        NaN        NaN        NaN        NaN   
2     Fuels- Bargarh(R)    75268.6    46382.6    77143.8    26767.4   
3  Fuels-Bhawanipatna(R    98617.5    48384.9    85663.9    34282.9   
4      Fuels-JEYPORE(R)     116454    77550.1    88960.6    37026.9   
5   Fuels-Jharsuguda(R)     123893    35085.5    89383.7    23636.5   
6     Fuels-ROURKELA(R)     297054    38585.2     264787    33459.8   
7   Fuels-SAMBALPUR (R)     113849    37967.9     108429    22160.5   
8        Keonjhar I RSA     105567    20263.9     101079    11532.6   
9       Keonjhar II RSA    21604.4    7816.54    11945.4       4756   

  2020-07-25 Unnamed: 8 2020-07-24 Unnamed: 6  
0         HS         MS         HS         MS  
1        NaN        NaN        NaN        NaN  
2    75558.1    22824.6    79148.1    35804.7  
3    94119.7      32859   

In [12]:
with pd.ExcelWriter(first_file, engine='openpyxl') as writer:
    writer.book = load_workbook(first_file)
    df.to_excel(writer, 'summary', index=False)

In [13]:
wb = load_workbook(first_file)
ws = wb['summary']
ws.delete_rows(3)
check = 0
for value in ws.iter_rows(min_row=1, max_row=1):
    for cell in value:
        if check % 2 == 0:
            cell.value = ""
        check += 1
        print(cell.value)
# print(df.columns.values)
print(ws['A3'].value)


2020-07-27

2020-07-26

2020-07-25

2020-07-24

Fuels- Bargarh(R)


In [14]:
"""
data = 'ABCDEFGHI'
for char in data:
    val = char + '1'
    print(ws[val].value)

print(ws.cell(row=1, column=2).value)

#Iterate through columns of the first row
for row in ws.iter_rows(min_row=1, max_row=1):
    for cell in row:
        print(cell.value)
"""

"\ndata = 'ABCDEFGHI'\nfor char in data:\n    val = char + '1'\n    print(ws[val].value)\n\nprint(ws.cell(row=1, column=2).value)\n\n#Iterate through columns of the first row\nfor row in ws.iter_rows(min_row=1, max_row=1):\n    for cell in row:\n        print(cell.value)\n"

In [15]:
from exchangelib import Account, Message, Credentials, HTMLBody
from exchangelib import Configuration, DELEGATE
import os

In [16]:
outlook_user = os.environ.get('OUTLOOK_USER')
outlook_password = os.environ.get('OUTLOOK_PASS')
outlook_server = os.environ.get('OUTLOOK_SERVER')
outlook_email = os.environ.get('OUTLOOK_EMAIL')

print(os.getenv('OUTLOOK_USER'))
print(os.getenv('OUTLOOK_PASS'))

ioc\00504802
Toptal@22


In [17]:
credentials = Credentials(username=outlook_user,
                         password=outlook_password
                         )
config = Configuration(server=outlook_server,
                      credentials=credentials)
account = Account(primary_smtp_address=outlook_email,
                 config=config,
                 autodiscover=False,
                 access_type=DELEGATE)

In [18]:
def create_html(ws):
    html = """
    <html><body><h3>Nozzle Sales Report:</h3>
    <table border=1>
    """
    # entering header data in html
    for whole_row in ws.iter_rows(min_row=1, max_row=2):
        html += "<tr>"
        for row in whole_row:
            html += f'<th>{row.value}</th>'
        html += "</tr>"
    # entering table data in html
    for whole_row in ws.iter_rows(min_row=3):
        html += "<tr>"
        for row in whole_row:
            if isinstance(row.value, float):
                html += f'<td>{round(row.value, 2)}</td>'
                print(round(row.value, 2))
            else:
                html += f'<td>{row.value}</td>'
        html += "</tr>"
    html += """
    </table>
    <p>
    <br>
    With Regards<br>
    Pankaj Barnwal<br>
    </p>
    This is an auto-generated mail.
    </body>
    </html>
    """
    return html

In [19]:
msg = Message(
    account=account,
    subject="Nozzle Sales Report - Sambalpur DO",
    body=HTMLBody(create_html(ws)),
    to_recipients=['barnwalp@indianoil.in']
)

msg.send_and_save()

75268.6
46382.57
77143.78
26767.45
75558.15
22824.56
79148.06
35804.67
98617.47
48384.95
85663.85
34282.9
94119.66
100737.95
44706.72
116454.36
77550.07
88960.59
37026.94
107657.13
40195.61
143278.1
66587.43
35085.53
89383.74
23636.54
103398.04
21275.22
105291.88
34178.19
297054.34
38585.19
264787.38
33459.79
269888.34
33402.66
280507.7
33828.76
113849.34
37967.95
108428.85
22160.51
125161.2
19261.4
114771.75
25908.89
105566.91
20263.91
101078.86
11532.63
125629.52
8563.87
115044.71
15381.97
21604.37
7816.54
11945.41
17227.53
4488.35
13784.78
6084.89
